In [ ]:
!ls drive/My\ Drive/Learning_material/Notebook/creditscoring

ls: cannot access 'drive/My Drive/Learning_material/Notebook/creditscoring': No such file or directory


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My\ Drive/Learning_material/Notebook/creditscoring

/content/drive/My Drive/Learning_material/Notebook/creditscoring


# START

In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
pd.options.display.float_format = '{:.2f}'.format

In [4]:
arpu_train = pd.read_csv('/content/drive/My Drive/project_credit_model/arpu_train.csv')
arpu_test = pd.read_csv('/content/drive/My Drive/project_credit_model/arpu_test.csv')

In [5]:
list_msisdn_of_test = list(arpu_test.msisdn)
list_msisdn_of_train = list(arpu_train.msisdn)

In [6]:
len(list_msisdn_of_test), len(list_msisdn_of_train)

(57445, 57721)

In [7]:
list_msisdn_of_test.extend(list_msisdn_of_train)

In [8]:
list_msisdn_of_train_test = list_msisdn_of_test

In [9]:
print(list_msisdn_of_train_test)

Output hidden; open in https://colab.research.google.com to view.

In [10]:
print(len(list_msisdn_of_train_test), len(set(list_msisdn_of_train_test)))

115166 115166


# Utils

In [11]:
def summary_missing(dataset):
  n_miss = dataset.isnull().sum()
  n_obs = dataset.shape[0]
  n_miss_per = n_miss/n_obs*100
  n_miss_tbl = pd.concat([n_miss, n_miss_per], axis = 1).sort_values(1, ascending = False).round(1)
  n_miss_tbl = n_miss_tbl[n_miss_tbl[1] != 0]
  print('No fields: ', dataset.shape[1])
  print('No missing fields: ', n_miss_tbl.shape[0])
  print('No values (both missing and non missing): ', dataset.shape[0])
  n_miss_tbl = n_miss_tbl.rename(columns = {0:'Number mising Value', 1:'Percentage missing Value'})
  return n_miss_tbl

In [12]:
def _number_unique_by_col(dataset):
  dict_table_nunique = {}
  for col in dataset.columns:
    dict_table_nunique[col] = dataset[col].nunique()
  return pd.Series(dict_table_nunique)

In [13]:
#Thống kê tỷ lệ (nhãn 0 : nhãn 1) theo các biến dự báo dạng object.
def _per_categorical(data, col):
  tbl_per = pd.pivot_table(data[['label', col]], index = ['label'], columns = [col], aggfunc = len)
  # print(tbl_per)
  # return
  per_categorical = (tbl_per.iloc[0, :]/tbl_per.iloc[1, :]).sort_values(ascending = True)
  # per_categorical = (tbl_per.iloc[0, :]/(tbl_per.iloc[0, :] + tbl_per.iloc[1, :])).sort_values(ascending = True)
  print(per_categorical)
  print('-------------------------------------\n')
  # return per_categorical

In [14]:
import matplotlib.pyplot as plt
def _plot_bar_classes(data, cols):
  data[cols].value_counts().plot.bar()
    # plt.figure(figsize = (20, 15))    
# for i in range(1, 15, 1):
#     plt.subplot(5, 3, i)
#     _plot_bar_classes(dtypes_object[i-1])
#     plt.title(dtypes_object[i-1])

In [15]:
def _tbl_dtype(dataset):
  sum_dtype = pd.DataFrame(dataset.dtypes).sort_values(0).rename(columns = {0:'Data Type'})
  return sum_dtype

In [16]:
#Vẽ biểu đồ tỷ lệ 1/0 (cho vay/ko cho vay) theo các biến dự báo dạng object.
def _plot_per_categorical(data, col):
    tbl_per = pd.pivot_table(data[['label', col]], index = ['label'], columns = [col], aggfunc = len)
    per_categorical = (tbl_per.iloc[0, :]/tbl_per.iloc[1, :]).sort_values(ascending = True)
    per_categorical.plot.bar()
    plt.title(col)
    return per_categorical

In [17]:
# Nhóm các giá trị rate gần bằng nhau vào 1 nhóm theo schedule_div.
def _devide_group(data, col, schedule_div = None, n_groups = 3, *kwargs):
  """
  Có 2 cách chia:
  (1) chia thành n_groups trong đó n-1 group đầu có số lượng phần tử (categories) bằng nhau
  và group cuối cùng là group thứ n có số phần tử >= số phần tử của mỗi group kia do nó chứa
  cả các categories bị lẻ ra khi chia thành các n_groups có số phần tử bằng nhau.
  (2) chia theo schedule_div hay chia theo "lịch lịch trình" mà ta đặt ra sao cho hợp lí khi nhìn vào dữ liệu.
  """
  cols = []
  tbl_per_cat = _per_categorical(data, col)

  if schedule_div is None:
    n_cats = len(tbl_per_cat)
    n_val_incat = int(n_cats/n_groups)
    n_odd = n_cats - n_groups*n_val_incat

    for i in range(n_groups):
      if i == n_groups - 1:
        # group cuoi se co so phan tu = n_cats + n_odds
        # tuc la van lay n_cats phan tu tiep theo va lay ca phan du n_odds (< n_cats)
        el = tbl_per_cat[(n_val_incat*i):(n_val_incat*(i+1)+n_odd)].index.tolist()
      else:
        el = tbl_per_cat[(n_val_incat*i):n_val_incat*(i+1)].index.tolist()    
      cols.append(el)
  else:
    idx = 0
    for n_cols in schedule_div:
        el_cols = tbl_per_cat[idx:(idx+n_cols)].index.tolist()
        cols.append(el_cols)
        idx += n_cols
  return cols

In [18]:
def _get_by_msisdn(data, msisdn):
  '''
  return copy of data by msisdn
  '''
  df = data.copy()
  return df[df.msisdn == msisnd]

# Merge table

In [19]:
# temp = pd.read_csv('temp_msisdn_in_train_test.csv')
# temp.sort_values(by=['msisdn','COL_2'], inplace=True)
# temp.COL_2 = pd.to_datetime(temp.COL_2, errors='coerce')

In [ ]:
# pd.to_datetime(temp.tail(100).COL_1.iloc[0:3], unit='m')

In [ ]:
# temp[temp.msisdn == '000100829f9d67fff3cc'].PARTNER_MSISDN.nunique()

In [ ]:
# sum(temp[temp.msisdn == '000100829f9d67fff3cc'].PARTNER_MSISDN.notna())

=> 1 thang sẽ liên lạc với 1 thằng nhiều lần trong tháng

# Create_feature

## Seconnd time 29/8

### TEMP FILE

#### Preprocessing

In [27]:
temp = pd.read_csv('/content/drive/My Drive/project_credit_model/temp.csv')
len(temp), temp.msisdn.nunique()

(32144442, 115166)

In [28]:
temp = temp[temp.msisdn.isin(list_msisdn_of_train_test)]
temp.to_csv('temp_msisdn_in_train_test.csv', index = False)
len(temp),temp.msisdn.nunique()

(32144442, 115166)

In [29]:
temp = pd.read_csv('temp_msisdn_in_train_test.csv')
len(temp)

32144442

In [30]:
temp.msisdn.nunique() == (arpu_train.msisdn.nunique() + arpu_test.msisdn.nunique())

True

In [31]:
temp.sort_values(by=['msisdn','COL_2'], inplace=True)

In [32]:
temp.head()

,msisdn,COL_1,COL_8,COL_11,PARTNER_MSISDN,COL_2,UPLOAD_DATA,COL_12
30130200,000100000eb52d2a47e1,4443341.00,0.00,588046.00,NaN,2019-10-01T00:07:58.000+07:00,833412.00,7.00
30130206,000100000eb52d2a47e1,4443341.00,0.00,5028933.00,NaN,2019-10-01T00:07:58.000+07:00,12910165.00,7.00
30130205,000100000eb52d2a47e1,4443341.00,15.00,0.00,00014db008ca447eed8f,2019-10-01T07:45:26.000+07:00,0.00,1.00
30130209,000100000eb52d2a47e1,4443343.00,0.00,719364.00,NaN,2019-10-01T07:45:48.000+07:00,2060576.00,7.00
30130203,000100000eb52d2a47e1,4443343.00,0.00,105869.00,NaN,2019-10-01T07:55:41.000+07:00,238075.00,7.00


#### EDA Temp

In [ ]:
temp = pd.read_csv('temp_msisdn_sort_msisdn_datetime.csv')
len(temp)

In [ ]:
temp.COL_2 = pd.to_datetime(temp.COL_2, errors='coerce')

#### Get part of day moring afternoon evening night

In [ ]:
def get_part_of_day(hour):
  return (
      np.nan if np.isnan(hour)
      else
      "morning" if 5 <= hour <= 11
      else
      "afternoon" if 12 <= hour <= 17
      else
      "evening" if 18 <= hour <= 22
      else
      "night"
  )

In [ ]:
temp[temp.COL_2.isna()].head(1).COL_2.dt.hour

In [ ]:
get_part_of_day(np.nan)

Theem feature trung binh mot ngay phat sinh bao nhieu cuoc goi
So cuoc goi phat sinh vao buoi sang
So cuoc goi phat sinh vao buoi chieu
So cuoc goi phat sinh ban dem

In [ ]:
# get COL_2 in hour
temp['COL_2_hour'] = temp.COL_2.dt.hour

In [ ]:
temp.head(10)

In [ ]:
temp["COL_2_part_of_day"] = temp.COL_2_hour.apply(get_part_of_day)

In [ ]:
num_calls_part_of_day = temp.groupby(['msisdn', 'COL_2_part_of_day']).agg(num_calls_part_of_day = ('PARTNER_MSISDN', pd.Series.count)).unstack(level = 1)

In [ ]:
num_calls_part_of_day

In [ ]:
num_calls_part_of_day.columns = ['_'.join(col).strip('_') for col in num_calls_part_of_day.columns]

In [ ]:
num_calls_part_of_day

In [ ]:
tesssssss = tesssssss.join(num_calls_part_of_day)

total call time by part of day (morning, afternoon, evening, night)

In [ ]:
total_call_time = temp[temp.PARTNER_MSISDN.notna()].groupby(['msisdn', 'COL_2_part_of_day']).agg(total_call_time_second_month = ('COL_8', 'sum')).unstack(level = 1)

In [ ]:
total_call_time.columns = ['_'.join(col).strip('_') for col in total_call_time.columns]

In [ ]:
total_call_time

In [ ]:
tesssssss = tesssssss.join(total_call_time)

In [ ]:
tesssssss.rename(columns={'total_calling_min_in_month':'total_call_time_second_month'}, inplace=True)

In [ ]:
tesssssss.loc['000100000eb52d2a47e1'].total_call_time_second_month == total_call_time.sum(axis=1).loc['000100000eb52d2a47e1']

Them feature tong thoi luong goi trong thang ung voi tung service

In [ ]:
temp

In [ ]:
total_call_time = temp[temp.PARTNER_MSISDN.notna()].groupby(['msisdn', 'COL_12']).agg(total_call_time_second_month = ('COL_8', 'sum')).unstack(level = 1)

In [ ]:
total_call_time.columns

https://stackoverflow.com/questions/10880813/typeerror-sequence-item-0-expected-string-int-found

TypeError: sequence item 0: expected string, int found
=> loi xay ra khi co phan tu la mot so float/int ma khong phai string khi goi join

string.join connects elements inside list of strings, not ints.

Use this generator expression instead :

values = ','.join(str(v) for v in value_list

In [ ]:
total_call_time.columns = ['_'.join(str(ele) for ele in col).strip('_') for col in total_call_time.columns]

In [ ]:
total_call_time

In [ ]:
tesssssss = tesssssss.join(total_call_time)

In [ ]:
tesssssss.loc['000100000eb52d2a47e1'].total_call_time_second_month == total_call_time.sum(axis=1).loc['000100000eb52d2a47e1']

Them feature so quan sat ma null toan bo ngoai tru msisdn

In [ ]:
temp_num_null_rows = temp.set_index('msisdn').isnull().groupby('msisdn').agg(temp_num_null_rows = ('COL_8','sum'))

In [ ]:
temp_num_null_rows

In [ ]:
tesssssss = tesssssss.join(temp_num_null_rows)

In [ ]:
tesssssss

In [ ]:
ls

In [ ]:
tesssssss

In [ ]:
tesssssss.fillna(0,inplace=True)

In [ ]:
tesssssss

In [ ]:
tesssssss.to_csv('temp_features_extraction_0002am_6_9_2020.csv')

In [21]:
tesssssss = pd.read_csv('/content/drive/My Drive/project_credit_model/temp_features_extraction_0002am_6_9_2020.csv')

Nghi ngo service 7 thi cac row gia tri COL_1 la theo  cap

In [22]:
temp[(temp.msisdn == '00010000e34bc5fff815') & (temp.COL_12 == 7)].COL_1.value_counts()

NameError: ignored

In [23]:
temp[(temp.msisdn == '00010000e34bc5fff815') & (temp.COL_12 == 7)].tail(3)

NameError: ignored

In [ ]:
temp[(temp.msisdn == '000100000eb52d2a47e1') & (temp.COL_12 == 7)].COL_1.value_counts()

In [24]:
temp[(temp.msisdn == '000100000eb52d2a47e1') & (temp.COL_12 == 7)].tail(11)

NameError: ignored

Có veervowis COL_12 = 7 thì các giá trị ơ COL_1 luôn theo cặp giá trị giống nhau

In [ ]:
temp[(temp.msisdn == '00010000e34bc5fff815') & (temp.COL_12 != 7)]

Nghe có vẻ côt 1 là tính tiền theo phút gọi cột 8 là không đúng nhìn tiền với phút gọi mà xem nhiều cái lệch vl mà vẫn có cùng tiền (nếu cột đó thực sự là tiền)

In [25]:
temp[(temp.msisdn == '00010000e34bc5fff815') & (temp.COL_12 != 7)].head(3)

NameError: ignored

Nếu cột COL_8 là phút gọi thì nó vô lí vl ạ. Nhìn vào 3 hàng trên =. cùng một ngày. Nếu COL_2 là thời điểm bắt đầu phát sinh "cuộc gọi" chẳng hạn thì từ hàng đầu là cho partner 0001d3179b6aaaa5f0f8 đén hàng sau là cho partner 000115d185ae0af597f6 thì COL_2 diff là 1 phút 51s Nếu COL_8 là thời gian và tính theo phút là không ỏn => Nếu có là thời gian thì nó là theo giây.

Còn có 
- 00010000e34bc5fff815	5204964.00	24.00	0.00	0001d3179b6aaaa5f0f8	2019-10-01 14:38:10+07:00	0.00	2.00

- 00010000e34bc5fff815	5204962.00	93.00	0.00	000137764a85959caa23	2019-10-30 20:17:58+07:00	0.00	2.00

- 00010000e34bc5fff815	5204961.00	9.00	0.00	000115d185ae0af597f6	2019-10-01 14:40:01+07:00	0.00	2.00


một cái là 24s, một cái là 93s thì đáng nhẽ cái 93s phải nhiều tiền hơn chứ => nhưng ngược lại lại tí hơn và lệch cũng không nhiều => Có thể cột đó không phải tiền.

Hay cái thứ 3 có 9h mà xấp xiwr "Tiền" cái 93s

=> thôi thì chưa đoán được thì cứ lấy trung bình thôi

In [ ]:
temp.COL_1.nunique()

In [ ]:
tesssssss

In [ ]:
summary_missing(temp)

tất cả đều có số lượng missing như nhau ngoại trừ  cột 1, cột partner

PARTNER_MSISDN 6194106

COL_1 733018

COL_8, COL_11, COL_2, COL_12, UPLOAD_DATA 577049

####Explore cac row ma COL_1 null cac cot con lai khong nul

In [26]:
temp_COL_1_na_other_notna = temp[(temp.COL_11.notna()) & (temp.COL_1.isna())]

NameError: ignored

In [ ]:
temp_COL_1_na_other_notna

In [ ]:
temp_COL_1_na_other_notna.COL_12.unique()

In [ ]:
temp_COL_1_na_other_notna[temp_COL_1_na_other_notna.COL_12 != 7.00]

####Explore cac row ma PARTNER_MSISDN null cac cot con lai khong nul

In [ ]:
temp[(temp.COL_11.notna()) & (temp.PARTNER_MSISDN.isna())]

In [ ]:
temp[(temp.COL_11.notna()) & (temp.PARTNER_MSISDN.isna())].COL_12.unique()

In [ ]:
temp[(temp.COL_11.notna()) & (temp.PARTNER_MSISDN.isna())].COL_8.unique()

In [ ]:
# # intersection two dataframe: can just perform a merge, this will use all columns and the default type of merge is inner so values must be present in both dfs:
# summary_missing(temp[(temp.COL_1.isna())].merge(temp[(temp.COL_11.isna())]))


In [ ]:
temp[temp.COL_1 == 0]

#### Explore COL_8 = 0

In [ ]:
temp.COL_8.value_counts()

In [ ]:
temp[temp.COL_8 == 0]

In [ ]:
temp.groupby(['msisdn', 'COL_1']).diff()

In [ ]:
temp[temp.COL_8 == 0].COL_12.unique()

In [ ]:
temp[(temp.COL_8 == 0) & (temp.COL_12 == 1)]

In [ ]:
temp[(temp.COL_8 == 0) & (temp.COL_12 == 2)]

In [ ]:
temp[(temp.COL_8 == 0) & (temp.COL_12 == 3)]

##### temp COL_12 = 1

In [ ]:
temp_COL12_1 = temp[temp.COL_12 == 1]

In [ ]:
summary_missing(temp_COL12_1)

In [ ]:
temp_COL12_1

In [ ]:
temp_COL12_1.COL_8.value_counts()

##### temp COL_12 = 2

In [ ]:
temp_COL12_2 = temp[temp.COL_12 == 2]

In [ ]:
summary_missing(temp_COL12_2)

In [ ]:
temp_COL12_2.COL_8.value_counts()

In [ ]:
temp_COL12_2

##### temp COL_12 = 3

In [ ]:
temp_COL12_3 = temp[temp.COL_12 == 3]

In [ ]:
summary_missing(temp_COL12_3)

In [ ]:
temp_COL12_3.COL_8.value_counts()

In [ ]:
temp_COL12_3

##### temp COL_12 = 7

In [ ]:
temp_COL12_7 = temp[temp.COL_12 == 7]

In [ ]:
summary_missing(temp_COL12_7)

In [ ]:
temp_COL12_7

In [ ]:
temp_COL12_7.COL_8.unique()

In [ ]:
temp_COL12_7.COL_11.describe()

In [ ]:
temp_COL12_7[temp_COL12_7.COL_11 == 0]

In [ ]:
# display discribe data nummeric by float format instead of scientific notation in pandas
# quick temporary: df.round(4)
# global: pd.options.display.float_format = '{:20,.2f}'.format
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
temp_COL12_7[temp_COL12_7.COL_11 == 0].UPLOAD_DATA.describe()

In [ ]:
temp_COL12_7[(temp_COL12_7.COL_11 == 0) & (temp_COL12_7.UPLOAD_DATA == 1018412999.0)]

In [ ]:
temp_COL12_7[temp_COL12_7.COL_1 == 0]

In [ ]:
temp[temp.COL_1 == 0]

In [ ]:
temp.COL_1.describe()

In [ ]:
temp[temp.COL_1 == 611.00]

In [ ]:
temp[(temp.COL_8 == 0) & (temp.COL_11 == 0) ]

COL_1 không có giá trị = 0 => chỉ có null hoặc khác 0 thôi

Nó đang ở kb?mb? có cần convert khoong?

Nó là data numeric không nhất thiết phải convert sang mb hay gb nhưng có thể cần norm hoăc standard

In [ ]:
temp_COL12_1.COL_11.unique(), temp_COL12_2.COL_11.unique(), temp_COL12_3.COL_11.unique(),temp_COL12_1.UPLOAD_DATA.unique(), temp_COL12_2.UPLOAD_DATA.unique(), temp_COL12_3.UPLOAD_DATA.unique()

* temp_COL12_7 có PARTNER_MSISDN có lượng missing data là 100% => vì service 7 là service dungf data nên ko có partner là đúng.

  Ngoài ra chỉ có COL_12_7 thì cột UPLOAD_DATA Và COL_11 mới tồn tại các giá trị khác 0 => 2 cột đó là cột download và upload của service 7

  Các cột khác trong temp_COL12_7 là COL_8 có giá trị là 0 toàn bộ => cột này đang dự đoán liên quan đến gọi điện mà cụ thể hơn là thời gian gọi => Ngoài ra COL_1 thì có giá trị khác 0 và cungx nhiều missing => chưa biết COL_1 là gì.

In [ ]:
temp_COL12_1.describe()

In [ ]:
temp_COL12_2.describe()

In [ ]:
temp_COL12_3.describe()

In [ ]:
temp_COL12_7.describe()

Đang dự đoán các giá trị 1,2,3,7 trong COL_12 tương ứng với các dịch vụ cước ứng với các thông tin được cung cấp trong my viettel alf

(gọi nội/ngoại mạng| nhắn tin nội/ngoại mạng| các dịch vụ GTGT khác | dịch vụ data) 
Tuy nhiên chỉ có gọi/ nhắn tin là có partner còn các dịch vụ GTGT khác và dịch vụ data không có partner? => mà các dịch vụ ứng với 1,2,3 đều có partner còn 7 không có partner

Đang dự COL_1, COL_2, COL_3, COL_7 

In [ ]:
# temp_count = temp.groupby('msisdn').agg(pd.Series.count)
# temp_count.to_csv('temp_groupby_msisdn_count.csv', index=False)

In [ ]:
temp_count

In [ ]:
temp[temp.msisdn == '000100000eb52d2a47e1']

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  print(temp[temp.msisdn == '0001ffff501a5473d949'])

In [ ]:
temp[temp.msisdn == '0001ffff501a5473d949'].to_csv('temp_0001ffff501a5473d949.csv', index = False)

In [ ]:
print(temp[temp.msisdn == '0001ffff501a5473d949'])

In [ ]:
temp_COL12_2

In [ ]:
temp.describe()

In [ ]:
_number_unique_by_col(temp)

In [ ]:
temp[temp.msisdn == '000100000eb52d2a47e1']

In [ ]:
group_by_msisdn = temp.groupby('msisdn')

In [ ]:
tbl = group_by_msisdn['COL_11', 'COL_12', 'UPLOAD_DATA', 'PARTNER_MSISDN'].agg(COL_11_month_mean = ('COL_11', 'mean'), UPLOAD_DATA_month_mean = ('UPLOAD_DATA', 'mean')
                , num_partner_contact_month = ('PARTNER_MSISDN', pd.Series.nunique), num_services = ('COL_12', pd.Series.nunique))

In [ ]:
tbl

In [ ]:
summary_missing(tbl)

In [ ]:
tbl.num_partner_contact_month.max()

In [ ]:
group_by_partner_msisdn = temp.groupby('PARTNER_MSISDN')

In [ ]:
tbl_partner = group_by_partner_msisdn['msisdn'].agg(num_be_called_by_partner_contact_month = ('msisdn', pd.Series.nunique))

In [ ]:
len(tbl_partner)

In [ ]:
tbl3 = tbl.join(tbl_partner)

In [ ]:
tbl3

In [ ]:
tbl3.num_be_called_by_partner_contact_month.max()

In [ ]:
sum(tbl3.num_be_called_by_partner_contact_month.notna())

In [ ]:
summary_missing(tbl3)

num_be_called_by_partner_contact_month có nan vì số số sdt được gọi đến (truy ngược lại) ít hơn nhiều so với số id gốc mssisn => có nan ở những chỗ không có thông tin ( chú ý lúc count từ partner sang mssisn thì không có nan nhé vì nunique sẽ tính là 0 nếu toàn bộ là nan (nó ignore nan nên số unique khi có nan = 0 => nó chỉ xuất hiện nan cột đó khi left_join mssisn với partner). Còn num_partner_contact_month không có nan vì nó count từ mssisn sang và id chính là mssisn luôn

=> co the xu li fillnan num_be_called_by_partner_contact_month với 1  hoặc ngược lại fill 0 của num_partner_contact_month với nan

có cái cả 2 table đều không đủ id của submission thì không chơi left join được mà chơi full outer join

In [ ]:
tbl3["num_be_called_by_partner_contact_month"] = tbl3["num_be_called_by_partner_contact_month"].fillna(0)

In [ ]:
summary_missing(tbl3)

In [ ]:
tbl3[tbl3.num_be_called_by_partner_contact_month > 0].num_be_called_by_partner_contact_month.max()

In [ ]:
tbl_total_calling_month = group_by_msisdn['COL_8'].sum(min_count = 1).to_frame(name='total_calling_min_in_month')

In [ ]:
tbl_total_calling_month

In [ ]:
summary_missing(tbl_total_calling_month)

In [ ]:
tbl4 = tbl3.join(tbl_total_calling_month)

In [ ]:
tbl4

tb_service_2 = pd.crosstab(temp['msisdn'],temp['COL_12'], dropna = True).rename_axis(None,axis=1).add_prefix('COL_12_')

dùng lệnh này không thôi sẽ bị mất các id mà các dịch vụ là null hết còn khonagr 114k row thôi
nên phải dùng
tb_service_2 = pd.crosstab(temp['msisdn'],temp['COL_12'].fillna('missing'), dropna = True).rename_axis(None,axis=1).add_prefix('COL_12_')

In [ ]:
tb_service_2 = pd.crosstab(temp['msisdn'],temp['COL_12'].fillna('missing'), dropna = True).rename_axis(None,axis=1).add_prefix('COL_12_')

In [ ]:
tb_service_2

Ở dưới sẽ chuẩn hóa nó về 0 1 hết để thành kiểu category. Người đó có sử dụng dịch vụ 1 là 1 không là 0 tương ứng với các cột còn lại => Tại sao phải làm thế dù đã có cột số dịch vụ sử dụng rồi => Vì có sự khác biệt giữa hai người cùng dùng 3 dịch vụ nhưng 3 dịch vụ của họ là 3 dịch vụ khác nhau ví dụ người A là 1 2 3 người B là 2 3 7 trong đó dịch vụ 7 sẽ được đánh giá cao hơn dịch vụ 1 trong việc xét credit scoring chẳng hạn.

NGoài ra cũng có thể giữ các giá trị trên để tính xem số giao dịch phát sinh đối với từng dịch vụ. Nó là sự khác biệt giữa hai người cùng dùng dịch vụ 7 chẳng hạn nhưng người A phát sinh tới 300 làn sử dụng dịch vụ 7 trong khi người B chỉ dùng 1 lần trong tháng.
Tuy nhiên cái này cần xem lại xem đúng là mỗi hàng có thêm ứng với từng người là do/liên quan đến cột dịch vụ không hay do cột khác và cột này không dính tới cột dịch vụ. => Nếu các dòng phát sinh có liên quan cột dịch vụ thì có thể có thêm các feature liên quan đến ngày tháng đến dịch vụ như tổng số ngày sử dụng dịch vụ 1 chẳng hạn (nó khác với tổng số lần phát sinh dịch vụ 1 nhé)\

Nếu liên quan đến cột khác ví dụ cột tiền thì có thể có feature thêm liên quan đến thời gian và tiền

Còn có thể có feature liên quan đến tổng số ngày có phát sinh một dịch vụ, hay cuộc gọi nào đó trong tháng

=> có thể null/ hay giá trị không cuarv các giá trị khác liên quan đến cột dịch vụ chẳng hạn. Nếu không phải dịch vụ 7 thì cột upload data/ download data sẽ là giá trị 0 còn là 7 thì sẽ có các giá trị khác 0

In [ ]:
tb_service_2.drop(columns=['COL_12_missing'], inplace=True)

In [ ]:
# %%timeit
# asss = pd.Series(np.where(tb_service_2['COL_12_1.0'] >= 1, 1, 0),
#           tb_service_2.index)

In [ ]:
# %%timeit
# tb_service_2.loc[tb_service_2['COL_12_1.0'] > 0, 'COL_12_1.0'] = 1

In [ ]:
for col in tb_service_2.columns:
  tb_service_2.loc[tb_service_2[col] > 0, col] = 1

In [ ]:
tb_service_2

In [ ]:
tbl5 = tbl4.join(tb_service_2)

In [ ]:
tbl5

In [ ]:
# tbl5.to_csv('temp_features_extraction_23_8_2020.csv')

In [ ]:
tesssssss = pd.read_csv('temp_features_extraction_23_8_2020.csv')

In [ ]:
tesssssss.set_index('msisdn', inplace=True)

In [ ]:
tesssssss.set_index('msisdn').join(arpu_train)

- còn có thể thêm feature về số ngày phát sinh gọi điện
- số cuộc gọi đi
- số cuộc gọi đên
(hiện tại có 2 cái khác cái tren là num_partner_contact_month là số số điện thoại liên lạc trong tháng (unique) và số số điện thoại (unique) được gọi đến trong tháng).


In [ ]:
summary_missing(tesssssss)

In [ ]:
temp.head()

#### confuse COL_8? Calling time? 

In [ ]:
temp.COL_8.value_counts()

In [ ]:
temp[(temp.COL_8.notna()) & (temp.COL_8 != 0)]

In [ ]:
temp[(temp.COL_8 == 0)]

In [ ]:
temp[(temp.PARTNER_MSISDN.notna())]

In [ ]:
temp[(temp.PARTNER_MSISDN.notna()) & (temp.COL_8 == 0)]

In [ ]:
25852587 + 96969 

In [ ]:
sum(temp[(temp.PARTNER_MSISDN.notna())].COL_8.isna())

### LOAN FILE

In [ ]:
# # remove mssidn not in train and test submission
# loan = pd.read_csv('loan_rm_COL_26.csv')
# loan = loan[loan.msisdn.isin(list_msisdn_of_train_test)]
# loan.to_csv('loan_rm_COL_26_msisdn_in_train_test.csv', index = False)

In [ ]:
loan = pd.read_csv('loan_rm_COL_26_msisdn_in_train_test.csv')

In [ ]:
loan.sample(10)

#### EDA LOAN FILE lại tí làm gì căng :)

In [ ]:
loan.describe()

In [ ]:
summary_missing(loan)

In [ ]:
loan.COL_23.unique(), loan.COL_23.nunique()

In [ ]:
loan[loan.COL_23 == 0].COL_24.unique()

In [ ]:
print(loan[loan.COL_23 == 1].COL_24.nunique())
print(list(loan[loan.COL_23 == 1].COL_24.unique()))

In [ ]:
_number_unique_by_col(loan)

In [ ]:
# loan[(loan.COL_23.notna()) & (loan.COL_24.notna()) & (loan.COL_25.notna())]
# # 3422458 + 178058 == 3600516

In [ ]:
# convert to datetime
loan.COL_25 = pd.to_datetime(loan.COL_25, errors='coerce')

In [ ]:
# sort by msisdn an datetime
loan.sort_values(by=['msisdn','COL_25'], inplace=True)

In [ ]:
# get all row at least one column in [COL_23, COL_24, COL_25] not nan
loan_not_na = loan[(loan.COL_23.notna()) | (loan.COL_24.notna()) | (loan.COL_25.notna())]

In [ ]:
loan_not_na

In [ ]:
loan_not_na.msisdn.nunique(), loan.msisdn.nunique()

In [ ]:
loan[loan.msisdn == '000100001be537fce5cf']

In [ ]:
loan.msisdn.value_counts()

In [ ]:
loan_not_na.msisdn.value_counts()

In [ ]:
(loan_not_na.msisdn.value_counts() > 2).sum()

In [ ]:
loan_not_na.msisdn.value_counts().plot.hist(30)

In [ ]:
loan_msisdn_00017eef45792c238bd4 = loan_not_na[loan_not_na.msisdn == '00017eef45792c238bd4']
loan_msisdn_00017eef45792c238bd4

In [ ]:
import matplotlib.pyplot as plt
loan_msisdn_00017eef45792c238bd4.set_index('COL_25', inplace=True)

In [ ]:
loan_msisdn_00017eef45792c238bd4

In [ ]:
loan_msisdn_00017eef45792c238bd4['day'] = loan_msisdn_00017eef45792c238bd4.index.day.values

In [ ]:
loan_msisdn_00017eef45792c238bd4

In [ ]:
loan_msisdn_00017eef45792c238bd4.COL_24.unique()

In [ ]:
# plt.style.available
# plt.style.use('fivethirtyeight')
# loan_msisdn_00017eef45792c238bd4.plot(figsize= (10,10))
# plt.show()

In [ ]:
loan_not_na.COL_24.value_counts()

In [ ]:
(loan_not_na.COL_24.value_counts()==1).sum()

In [ ]:
# loan_not_na.groupby('msisdn')['msisdn'].agg(count_row = ('msisdn', 'count')).sort_values('count_row')
loan_not_na.msisdn.value_counts()

In [ ]:
# tổng số giao dịch vào (tổng số số 1) 
# tổng số giao dịch ra (tổng số số 0) trong tháng
# tổng số lần thực hiện giao dịch trong tháng (tổng của 1 và 0)

# tổng tiền vào
# tổng tiền ra
# tổng chênh lệch tiền vào ra trong tháng (phải phân ra 3 cái thay vì chỉ dùng tổng chênh lệch tiền vào ra trong tháng  để phân biệt một người
#   2 nguoi co cung tong chenh lech vao ra la 10 nhung mot nguoi vao la 100000 ra la 99990 con nguoi con lai la vao la 90 ra 80 => hai nguoi nay se co tiem nang khac nhau)

# tổng số ngày co phat sinh giao dịch trong tháng


# Có thể thêm
# tổng số tuần có phát sinh giao dịch trong tháng


In [ ]:
loan.COL_25.max(), loan.COL_25.min()

In [ ]:
loan.COL_25.max() - loan.COL_25.min()

In [ ]:
time_transaction_each_msisdn = loan_not_na.groupby(by=['msisdn']).agg(col_25_min=('COL_25',np.min),
             col_25_max=('COL_25',np.max))

In [ ]:
time_transaction_each_msisdn

In [ ]:
time_transaction_each_msisdn = loan.groupby(by=['msisdn'])['COL_25'].agg([min, max]).add_prefix('COL_25_')

In [ ]:
time_transaction_each_msisdn

In [ ]:
time_transaction_each_msisdn_max_min = time_transaction_each_msisdn.COL_25_max - time_transaction_each_msisdn.COL_25_min

In [ ]:
time_transaction_each_msisdn_max_min.value_counts()

In [ ]:
loan_not_na[loan_not_na.COL_23 == 0]

In [ ]:
loan_not_na[loan_not_na.COL_23 == 0].COL_24.value_counts()

In [ ]:
loan_not_na[loan_not_na.COL_23 == 1].COL_24.value_counts()

In [ ]:
sum(loan_not_na[loan_not_na.COL_23 == 1].COL_24.value_counts()>4)

In [ ]:
loan.COL_25.min(), loan.COL_25.max()

In [ ]:
loan_not_na.msisdn.nunique(), loan.msisdn.nunique()

=> Có những người chưa vay/ không được vay trong tháng

#### Giờ thì tạo feature từ LOAN file nafy @@

Tạo feature sau ngày 24/8

In [ ]:
loan_features = pd.read_csv('loan_features_extraction_24_8_2020.csv')

In [ ]:
loan_features

In [ ]:
loan_features[loan_features.msisdn == '00010003ac76910ee3fc']

Tổng số giao dịch vào (tổng số số 1)

Tổng số giao dịch ra (tổng số số 0) trong tháng

Tổng số lần thực hiện giao dịch trong tháng (tổng của 1 và 0)
=> Người nào không được vay/ không vay và trả nợ trong tháng này sẽ có tổng số lần thực hiện giao dịch = 0

Tổng tiền trả, tổng tiền vay
Tổng chênh lệch tiền vào ra trong tháng (phải phân ra 3 cái thay vì chỉ dùng tổng chênh lệch tiền vào ra trong tháng  để phân biệt một người
=> < 0 là chưa trả hết nợ; >0 chắc là tiền vay và tiền lãi hoặc tiền đọng còn lại tháng trước vay nữa -> tóm lại là khả năng cao trả hết nợ rồi; =0 có thể trả hết nợ khoặc như nói ở tren không vay không trả hoặc chưa có tiền trả (từ tháng trước đó ?) 

Tổng số ngày phát sinh giao dịch vay trả trong tháng

MỚI:
Số giao dịch lớn nhất trong 1 ngày

Số giao dịch nhỏ nhất trong 1 ngày (cái này có vẻ không có tác dụng lắm => hầu hết đều là 1)

Số giao dịch vào lớn nhất trong 1 ngày (cái này sợ cũng không có nhiều khác biệt)

Số giao dịch ra lớn nhất trong 1 ngày. (cái này cũng sơ không có nhiều khác biệt)

Vẫn là những cái trên nhưng là trong 1 tuần. 


Số tuần có phát sinh giao dịch (<=4)

Một ngày vào max bao nhiêu tiền.

Một ngày ra max bao nhiêu tiền.

Một ngày tổng vào ra max bao nhiêu tiền

Vẫn là những cái trên nhưng theo tuần. 


- Số dòng null

In [ ]:
loan['day_COL_25'] = loan.COL_25.dt.day

In [ ]:
loan.sample(10)

In [ ]:
loan_num_trans_each_service_per_day = loan.groupby(['msisdn', 'day_COL_25', 'COL_23']).COL_24.count().unstack(level = 2)

In [ ]:
loan_max_trans_0_in_one_day = loan_num_trans_each_service_per_day.groupby(['msisdn'])[0.00].max(numeric_only = True).to_frame(name='loan_max_trans_0_in_one_day')

In [ ]:
loan_max_trans_1_in_one_day = loan_num_trans_each_service_per_day.groupby(['msisdn'])[1.00].max(numeric_only = True).to_frame(name='loan_max_trans_1_in_one_day')

In [ ]:
loan_min_trans_0_in_one_day = loan_num_trans_each_service_per_day.groupby(['msisdn'])[0.00].min(numeric_only = True).to_frame(name='loan_min_trans_0_in_one_day')

In [ ]:
loan_min_trans_1_in_one_day = loan_num_trans_each_service_per_day.groupby(['msisdn'])[1.00].min(numeric_only = True).to_frame(name='loan_min_trans_1_in_one_day')

In [ ]:
loan_features_2 = loan_features.set_index('msisdn')

In [ ]:
# Them du lieu cho loan feature
loan_features_2 = loan_features_2.join(loan_max_trans_0_in_one_day).join(loan_max_trans_1_in_one_day)
loan_features_2 = loan_features_2.join(loan_min_trans_0_in_one_day).join(loan_min_trans_1_in_one_day).fillna(0)

In [ ]:
loan_mean_trans_0_in_one_day = loan_num_trans_each_service_per_day.groupby(['msisdn'])[0].mean().to_frame(name='loan_mean_trans_0_in_one_day')

In [ ]:
loan_mean_trans_1_in_one_day = loan_num_trans_each_service_per_day.groupby(['msisdn'])[1].mean().to_frame(name='loan_mean_trans_1_in_one_day')

In [ ]:
# loan_mean_trans_in_one_day = loan_num_trans_each_service_per_day.groupby(['msisdn']).mean()
# loan_mean_trans_in_one_day

In [ ]:
loan_num_trans_per_day  = loan_num_trans_each_service_per_day.sum(axis=1).to_frame(name = 'loan_num_trans_per_day')

In [ ]:
loan_mean_num_trans_one_day = loan_num_trans_per_day.groupby('msisdn').mean().rename(columns = {'loan_num_trans_per_day':'loan_mean_num_trans_one_day'})

In [ ]:
# Them du lieu cho loan feature
loan_features_2 = loan_features_2.join(loan_mean_trans_0_in_one_day).join(loan_mean_trans_1_in_one_day).join(loan_mean_num_trans_one_day).fillna(0)

In [ ]:
loan_mean_num_trans_one_day[loan_mean_num_trans_one_day.loan_mean_num_trans_one_day == 5]

In [ ]:
loan[loan.msisdn == '00010da0296953493690']
# chu y co nhung id nhu 00010da0296953493690 co mean mean_num_trans_one_day lon va = 5 nhung thuc te = 5 la ung voi 1 ngay giao dich duy nhat
# chua chac da bang nhung nguoi co mean = 3 nhung thuc hien giao dich trong nhieu ngay

In [ ]:
loan[loan.msisdn == '0001007946c6661ab5b1']

=> Có những người một ngày thực hiện cả nhieu lenh (ra vao) như 0001007946c6661ab5b1

In [ ]:
loan.groupby(['msisdn', 'day_COL_25', 'COL_23']).COL_24.sum()

In [ ]:
loan_features_2

Lấy ra số tiền vào max min (|| là max) trong 1 ngày

In [ ]:
loan_sum_money_per_day_service = loan.groupby(['msisdn', 'day_COL_25', 'COL_23']).COL_24.sum().unstack(level=2)

In [ ]:
loan_sum_money_per_day_service

In [ ]:
loan_max_sum_money_per_day_service = loan_sum_money_per_day_service.groupby('msisdn').max().rename(columns = {0.00: 'loan_max_sum_money_per_day_0', 1.00:'loan_max_sum_money_per_day_1'})

In [ ]:
loan_min_sum_money_per_day_service = loan_sum_money_per_day_service.groupby('msisdn').min().rename(columns = {0.00: 'loan_min_sum_money_per_day_0', 1.00:'loan_min_sum_money_per_day_1'})

In [ ]:
loan_mean_sum_money_per_day_service = loan_sum_money_per_day_service.groupby('msisdn').mean().rename(columns = {0.00: 'loan_mean_sum_money_per_day_0', 1.00:'loan_mean_sum_money_per_day_1'})

In [ ]:
loan_features_2 = loan_features_2.join(loan_max_sum_money_per_day_service).join(loan_min_sum_money_per_day_service).join(loan_mean_sum_money_per_day_service)

In [ ]:
# loan_features_2.to_csv('loan_features_extraction_0012am_04092020.csv')

In [ ]:
loan.sample(10)

add feature number null for loan

In [ ]:
loan_num_null_rows = loan.set_index('msisdn').isnull().groupby('msisdn').agg(loan_num_null_rows = ('COL_24','sum'))

In [ ]:
loan_num_null_rows

In [ ]:
loan_feature_2_old = pd.read_csv('loan_features_extraction_0012am_04092020.csv')

In [ ]:
loan_feature_2_old.set_index('msisdn', inplace= True)

In [ ]:
loan_feature_2_old = loan_feature_2_old.join(loan_num_null_rows)

In [ ]:
loan_feature_2_old

In [ ]:
loan_feature_2_old.to_csv('loan_features_extraction_06092020_ver3.csv')

Có thể chuyển hết số âm và trị tuyệt đối => ròi lấy max => bản chất là tiền ?dương mà

##### idea: 
Chú ý mean mỗi giao dịch khác với mean tiền 1 ngày/(n ngày phát sinh giao dịch) của mỗĩ người nhé.

Đếm số lượng unique của cột tiền vào ??? có giúp ích không nhỉ.

Bảng recharge có thể đếm số loại mệnh giá thẻ napj?


Trước 24/8

In [ ]:
#

In [ ]:
loan.COL_25 = pd.to_datetime(loan.COL_25, errors='coerce')

In [ ]:
loan.sort_values(by=['msisdn','COL_25'], inplace=True)

In [ ]:
loan.sample(10)

In [ ]:
loan.COL_25.dt.day.unique()

In [ ]:
loan.COL_23.value_counts()

In [ ]:
tbl_num_transaction_in_out = pd.crosstab(loan['msisdn'],loan['COL_23'].fillna('missing'), dropna = True).rename_axis(None,axis=1).add_prefix('num_money_month_COL23_')

In [ ]:
tbl_num_transaction_in_out

In [ ]:
tbl_num_transaction_in_out['num_transaction_in_month_COL23'] = tbl_num_transaction_in_out['num_money_month_COL23_0.0'] + tbl_num_transaction_in_out['num_money_month_COL23_1.0']

In [ ]:
tbl_num_transaction_in_out

In [ ]:
tbl_num_transaction_in_out.drop(columns=['num_money_month_COL23_missing'], inplace= True)

In [ ]:
tbl_num_transaction_in_out

In [ ]:
tbl_sum_money_out = loan[loan.COL_23 == 0].groupby('msisdn').sum().add_prefix('sum_money_month_out_')

In [ ]:
tbl_sum_money_in = loan[loan.COL_23 == 1].groupby('msisdn').sum().add_prefix('sum_money_month_in_')

In [ ]:
tbl_sum_money_in.drop(columns=['sum_money_month_in_COL_23'], inplace= True)

In [ ]:
tbl_sum_money_out.drop(columns=['sum_money_month_out_COL_23'], inplace= True)

In [ ]:
tbl_sum_money_out

In [ ]:
tbl_sum_money_in

In [ ]:
# ttt = pd.merge(tbl_sum_money_in,tbl_sum_money_out,on='msisdn',how='outer')

In [ ]:
# ttt.loc['0001fffe7b7a8828c84d']

In [ ]:
# tbl_sum_money_in_out_month_COL_23 = tbl_sum_money_in.join(tbl_sum_money_out, how='outer')

In [ ]:
# tbl_sum_money_in_out_month_COL_23

In [ ]:
tbl_LOAN = tbl_num_transaction_in_out.join(tbl_sum_money_in)

In [ ]:
tbl_LOAN = tbl_LOAN.join(tbl_sum_money_out)

In [ ]:
tbl_LOAN

In [ ]:
tbl_LOAN.sum_money_month_in_COL_24.fillna(0, inplace=True)
tbl_LOAN.sum_money_month_out_COL_24.fillna(0, inplace=True)

In [ ]:
tbl_LOAN

In [ ]:
tbl_LOAN['sum_money_in_out_COL_24'] = tbl_LOAN.sum_money_month_in_COL_24 + tbl_LOAN.sum_money_month_out_COL_24

In [ ]:
tbl_LOAN

Chú ý nếu + hai column của 2 dataframe nếu 1 row của column này là null thì dù row tương ứng của nó ở column tương ứng của nó ở column kia có là null hay không thì nó vẫn cho kết quả + là null.
Nên nếu muốn + hai column thì phải xử lí trước ví dụ fill null là 0 trước.

chạy thử cái này để so sánh xem sau xử lí tính đúng chưa
loan.groupby('msisdn').COL_24.sum()
tính sum_in_out dùng luôn được lệnh này tuy nhiên vẫn muốn tính dựa vào 2 cột kia để tiện thể check luôn 2 cột kia đã tổng hợp đúng chưa.

In [ ]:
sum(loan.groupby('msisdn').COL_24.sum().values == tbl_LOAN.sum_money_in_out_COL_24.values)

In [ ]:
loan.COL_25 = pd.to_datetime(loan.COL_25, errors='coerce')

In [ ]:
loan.COL_25.dtype

In [ ]:
loan.COL_25.max(), loan.COL_25.min()

Thời gian thu thập đúng trong một tháng nên có thể lấy nguyên 'day' ra để count số ngày phát sinh giao dịch (nếu không đúng trong 1 tháng thì phải xử lí trước mới tính được vì có trường hợp cùng là 'day' 1 nhưng là của 2 tháng khác nhau

In [ ]:
# loan.COL_25.dt.day.value_counts()

In [ ]:
loan['day_COL_25'] = loan.COL_25.dt.day

In [ ]:
tb_num_days_trans_arise_month_COL_25 = loan.groupby('msisdn')['day_COL_25'].agg(num_days_trans_arise_month_COL_25 = ('day_COL_25', pd.Series.nunique))

In [ ]:
tbl_LOAN_2 = tbl_LOAN.join(tb_num_days_trans_arise_month_COL_25)

In [ ]:
tbl_LOAN_2

Ngoài ra có thểm thêm các feature liên quan đến thời gian như các bảng khác tuy nhiên do tính chất của bảng nên nó sẽ hơi khác.
Ví dụ chả ai tính số ngày phát sinh vay/ trả cả mà có thẻ là số ngày từ vay đến trả chẳng hạn.

In [ ]:
summary_missing(tbl_LOAN_2)

In [ ]:
# tbl_LOAN_2.to_csv('loan_features_extraction_24_8_2020.csv')

In [ ]:
loan_features_extraction_24_8_2020 = pd.read_csv('loan_features_extraction_24_8_2020.csv')

In [ ]:
loan_features_extraction_24_8_2020

### RECHARGE FILE

In [ ]:
# # remove mssidn not in train and test submission
# recharge = pd.read_csv('recharge.csv')
# recharge = recharge[recharge.msisdn.isin(list_msisdn_of_train_test)]
# recharge.to_csv('recharge_msisdn_in_train_test.csv', index = False)

In [ ]:
recharge = pd.read_csv('recharge_msisdn_in_train_test.csv')

In [ ]:
recharge.sample(5)

In [ ]:
recharge

In [ ]:
recharge[recharge.COL_31.notna()]

In [ ]:
recharge.msisdn.nunique()

#### Lại EDA recharge tí làm gì căng

In [ ]:
summary_missing(recharge)

In [ ]:
recharge.COL_31.value_counts()

Có số tiền lẻ nạp được lẻ à? Nạp được qua Viettel Pay

In [ ]:
recharge[recharge.COL_30 == 'C'].COL_31.value_counts()

In [ ]:
recharge[recharge.COL_30 == 'V'].COL_31.value_counts()

dummies variable trap
3 categories => chỉ cần biểu diễn bằng 2 feature onehotvector là đủ
=> Đúng nhưng chỉ trong trường hợp là thực sự "onehot" tức là 1 cái là 1 thì còn lại là 0 tất.

Còn mà loại 1 observation có thể có cả 2 hoặc cả 3 loại category thì phải biểu diễn đúng bằng 3 category.


#### chơi

In [ ]:
recharge.COL_29 = pd.to_datetime(recharge.COL_29, errors='coerce')

In [ ]:
recharge.sort_values(by=['msisdn','COL_29'], inplace=True)

In [ ]:
recharge_in_train = recharge[recharge.msisdn.isin(list_msisdn_of_train)]

1. LQ1

số dịch vụ (C,V) mà id dùng

Số giao dịch phát sinh trong tháng dịch vụ C

Số giao dịch phát sinh trong tháng dịch vụ V

Tổng số giao dịch recharge phát sinh trong tháng (cả C,V)

Trung bình một tuần trong tháng phát sinh bao nhiêu giao dịch (cả C,V)

2. LQ thời gian

Số ngày phát sinh giao dịch (cả C,V) trong tháng.

Số tuần phát sinh giao dịch (cả C,V) trong tháng

3. LQ đến tiền
 
Tổng số tiền phát sinh do giao dịch C trong tháng

Tổng số tiền phát sinh do giao dịch V trong tháng

Tổng số tiền phát sinh giao dịch trong tháng (cả C,V)


Trung bình theo tuần tiền/ dịch vụ ???????????

4. Trung bình số tiền sử dụng trong tháng (chia trung bình cho số lần phát sinh tiền thay vì 30 (30 ngày)
=> chính là trung bình dùng bao nhiêu tiền cho 1giao dich

Ngược lại ta cũng có trung bình số tinề cho C và V

=> cái này có bị trùng với tổng số tiền không nhỉ vì bản chất là lấy tổng chia cho hằng số (số giao dich)

Có thể phân biệt 1 người chỉ nạp 1 lần 1 tỉ, 1 người nạp 10 lần mới được 1 tỉ
(bằng cái trung bình trên hoặc bằng tổng tiền và số lần phát sinh giao dịch)
=> thay bằng max(C), max(V)

In [ ]:
recharge_feature_25 = pd.read_csv('recharge_features_extraction_25_8_2020.csv')

In [ ]:
recharge_feature_25.set_index('msisdn', inplace= True)

In [ ]:
recharge_feature_25

In [ ]:
recharge_feature_25[(recharge_feature_25.COL_30_C != 0) & (recharge_feature_25.COL_30_V != 0)]

In [ ]:
recharge_COL_30_unique_by_msisdn = recharge.groupby('msisdn')['COL_30'].agg(num_services_COL_30 = ('COL_30', pd.Series.nunique))

In [ ]:
recharge_COL_30_unique_by_msisdn

In [ ]:
recharge_COL_30_unique_by_msisdn.num_services_COL_30.value_counts()

Vậy là có cái có cả C và V

Create feature: number of services (C|V|C,V|none|) per user (0,1,2)

In [ ]:
recharge_feature_25 = recharge_feature_25.join(recharge_COL_30_unique_by_msisdn)

In [ ]:
recharge_feature_25

Create feature max/min rechare 'C', 'V' in month ( ngoài ra còn có thể chơi kiểu max min theo từng tuần)

In [ ]:
recharge_max_money_C_COL_31 = recharge[recharge.COL_30 == 'C'].groupby(['msisdn']).agg(max_money_C_COL_31 = ('COL_31', max))
recharge_max_money_V_COL_31 = recharge[recharge.COL_30 == 'V'].groupby(['msisdn']).agg(max_money_V_COL_31 = ('COL_31', max))

In [ ]:
recharge_min_money_C_COL_31 = recharge[recharge.COL_30 == 'C'].groupby(['msisdn']).agg(min_money_C_COL_31 = ('COL_31', min))
recharge_min_money_V_COL_31 = recharge[recharge.COL_30 == 'V'].groupby(['msisdn']).agg(min_money_V_COL_31 = ('COL_31', min))

In [ ]:
recharge_feature_25 = recharge_feature_25.join(recharge_max_money_C_COL_31).join(recharge_max_money_V_COL_31)

In [ ]:
recharge_feature_25 = recharge_feature_25.join(recharge_min_money_C_COL_31).join(recharge_min_money_V_COL_31)

In [ ]:
recharge_feature_25.update(recharge_feature_25[['max_money_C_COL_31','max_money_V_COL_31']].fillna(0))

In [ ]:
recharge_feature_25.update(recharge_feature_25[['min_money_C_COL_31','min_money_V_COL_31']].fillna(0))

Create feature max money used for service recharge (both C and V)

from get max of two column max_money_C_COL_31 and max_money_V_COL_31

In [ ]:
recharge_feature_25['max_money_C_V_COL_31'] = recharge_feature_25[['max_money_C_COL_31', 'max_money_V_COL_31']].max(axis=1)

min thì hơi dị hơn một tí, min thì phải lấy min > 0 ( vì một cái = 0 tương ứng với null không có dữ liệu => thực tế min nạp là số > 0 nếu có)

In [ ]:
recharge_feature_25['min_money_C_V_COL_31'] = recharge_feature_25[['min_money_C_COL_31', 'min_money_V_COL_31']].where(recharge_feature_25[['min_money_C_COL_31', 'min_money_V_COL_31']].gt(0)).min(axis = 1)

In [ ]:
recharge_feature_25[(recharge_feature_25.max_money_C_COL_31 > 0) & (recharge_feature_25.max_money_V_COL_31 > 0)]

In [ ]:
recharge_feature_25[(recharge_feature_25.max_money_C_COL_31 == 0) | (recharge_feature_25.max_money_V_COL_31 == 0)]

In [ ]:
recharge_feature_25[(recharge_feature_25.max_money_C_COL_31 == 0) & (recharge_feature_25.max_money_V_COL_31 == 0)]

In [ ]:
recharge_feature_25.update(recharge_feature_25['min_money_C_V_COL_31'].fillna(0))

In [ ]:
recharge_feature_25[(recharge_feature_25.max_money_C_COL_31 == 0) & (recharge_feature_25.max_money_V_COL_31 == 0)]

In [ ]:
recharge_feature_25[(recharge_feature_25.max_money_C_COL_31 == 0) | (recharge_feature_25.max_money_V_COL_31 == 0)]

Nhân thấy mỗi user (msisdn - sdt) đều có các rows là null tất các columns (ngoại trừ msisdn nhé - tất nhiên rồi :) ) và số null row của các user là khác nhau => rất có thể không phải là ngẫu nhiên mà số null row (tất cả column) của các user là khác nhau => khả năng cao là các null row này ứng với một số cột feature nào đó đã bị drop đi => có thể "tận dụng" các feature "ẩn" này = cách tạo một feature số null rows (tất cả column) của từng msisdn.

In [ ]:
recharge_num_null_rows = recharge.set_index('msisdn').isnull().groupby('msisdn').agg(recharge_num_null_rows = ('COL_29','sum'))

In [ ]:
recharge_num_null_rows

In [ ]:
recharge_num_null_rows['recharge_num_null_rows'].value_counts()

In [ ]:
recharge_feature_25 = recharge_feature_25.join(recharge_num_null_rows)

In [ ]:
recharge_feature_25[(recharge_feature_25.COL_30_C == 1) & (recharge_feature_25.COL_30_V == 1)]

Bình thường có thể drop COL_30_V vì COL_30_C và COL_30_V nếu 2 cột này tao thành "one-hot" thực sự nhưng nó không phải one-hot vì có trường hợp cả 2 đều có thể là 1 ( người dùng 2  dịch vụ)

Vậy nếu 2 cột trên đều là 1 1 tức là họ dùng 2 dịch vụ. Còn là 1 0 hoặc 0 1 tức là chỉ dùng 1 trong hay => thế thì cần gì cột số dịch vụ họ dùng là recharge_num_null_rows nữa nhỉ => nó đủ hàm nghĩa nếu dùng 2 cột kia rồi.

Cứ để đó lúc train mô hình nếu cần có thể loại sau vì biết đâu chỉ cần cột recharge_num_null_rows là đủ tốt cho model rồi không cần thiết phải rạch ròi từng loại dùng COL_30_C và COL_30_V => lúc đó có thể còn drop 2 cột này thay vì recharge_num_null_rows

#####Ideas 0012am 3/9/2020: 
- Có cần feature kiểu nạp đầu tháng + nạp cuối tháng hay nạp vào dịp lễ + nạp vào cuối tuần...không nhỉ  :)))
=> Thói quen của người dùng => Biết đâu đấy những người hay được cho vay credit là những người hay có thói quen nạp tiền vào cuối tuần vì một lí do nào đó thì sao
=> có ngày tháng làm được nhiều thứ phết @@

- Ngoài ra còn idea kiểu merge ngày tháng các bảng với nhau => biết đâu các bảng này các feature lại có mlh gì đó liên quan đến ngày tháng như kiểu người này gọi điện mất nhiề tiền => làm tài khoản cạn tiền (bảng temp) thì mới nạp tiền (bảng recharge) khác với người thường xuyên nạp định kì hàng tháng, người thừowng xuyên liên lạc nạp tiền;

- Có thời gian đúng không -> có thể tính diff thời gian nạp giữa các lần nạp của 1 msisdn. (sort theo thời gian ròi lấy diff)
=> Tính min/max giữa các lần nạp liên tiếp.

- đã Created feature max/min rechare 'C', 'V' in month ( ngoài ra còn có thể chơi kiểu max min theo từng tuần) 

=> hay thói quen nạp tiền vào tuần thứ mấy trong tháng ( tuần 1,2,3,4) 

- Đã có feature max cho một loại giao dịch nạp trong tháng rồi
Có thể thêm feature nạp max (tổng nạp max) trong 1 ngày.

In [ ]:
recharge_feature_25

In [ ]:
# recharge_feature_25.to_csv('recharge_features_extraction_0055am_03092020.csv')

In [ ]:
recharge_feature_25

Tạo các feature trước ngày 2582020

In [ ]:
recharge[recharge.COL_30 == 'C'].COL_31.value_counts()

In [ ]:
recharge[recharge.COL_30 == 'V'].COL_31.value_counts()

Có vẻ có 2 hình thức nạp thẻ có thể một hình thức dạng như nạp thẻ online như ViettelPay thì có tiền lẻ => tag 'C', còn hình thức nạp thẻ thông thường không có tiền kiểu lẻ lẻ ra thì là tag 'V'

In [ ]:
tb_recharge_service = pd.crosstab(recharge['msisdn'],recharge['COL_30'].fillna('missing'), dropna = True).rename_axis(None,axis=1).add_prefix('COL_30_')

In [ ]:
tb_recharge_service

In [ ]:
tb_recharge_service.drop(columns=['COL_30_missing'], inplace=True)

In [ ]:
tb_recharge_service_2 = tb_recharge_service.copy().add_prefix('num_arise_in_month_')

In [ ]:
tb_recharge_service_2

In [ ]:
for col in tb_recharge_service.columns:
  tb_recharge_service.loc[tb_recharge_service[col] > 0, col] = 1

In [ ]:
tb_recharge_service = tb_recharge_service.join(tb_recharge_service_2)

In [ ]:
summary_missing(tb_recharge_service)

Các num_arise_in_month_COL_30_C	num_arise_in_month_COL_30_V đều không có giá trị null nên dùng operator + để tính tổng giá trị hai cột được

In [ ]:
tb_recharge_service['num_trans_arise_month_COL_30'] = tb_recharge_service['num_arise_in_month_COL_30_C'] + tb_recharge_service['num_arise_in_month_COL_30_V']

In [ ]:
tb_recharge_service

In [ ]:
tb_recharge_service.num_trans_arise_month_COL_30.max()

In [ ]:
recharge['week_of_year'] = recharge.COL_29.dt.weekofyear

In [ ]:
recharge['day_in_month'] = recharge.COL_29.dt.day

In [ ]:
recharge

In [ ]:
tb_num_days_trans_arise_month_COL_30 = recharge.groupby('msisdn')['day_in_month'].agg(num_days_trans_arise_month_COL_30 = ('day_in_month', pd.Series.nunique))

In [ ]:
tb_num_days_trans_arise_month_COL_30.num_days_trans_arise_month_COL_30.unique()

In [ ]:
tb_recharge_service = tb_recharge_service.join(tb_num_days_trans_arise_month_COL_30)

In [ ]:
tb_recharge_service

Ngoài ra có thể có feature số ngày phát sinh giao dịch V, số ngày phát sinh giao dịch C (Chú ý 1 ngày có thể có thể phát sinh cả hai giao dịch C và V)
=> Tuy nhiên có thể phần này tính chất này đã được thể hiên qua tương quan số giao dịch C, số giao dịch V trong tháng rồi 
=> Note có thể xem xét cho thêm

In [ ]:
tb_recharge_service.loc[tb_recharge_service.num_days_trans_arise_month_COL_30 == 26]

In [ ]:
recharge.loc[recharge.msisdn == '0001de11ab2bdf27a469']

In [ ]:
recharge.week_of_year.value_counts()

In [ ]:
recharge.groupby('msisdn').week_of_year.nunique()

In [ ]:
recharge.groupby('msisdn').week_of_year.nunique().unique()

In [ ]:
tb_recharge_service['num_week_arise_trans_C_V_COL_30'] = recharge.groupby('msisdn').week_of_year.nunique()

In [ ]:
tb_recharge_service

In [ ]:
recharge.groupby(['msisdn', 'week_of_year'])['COL_30'].count().unique()

In [ ]:
recharge.groupby(['msisdn', 'week_of_year'])['COL_30'].value_counts().unique()

Ngoài ra có thể thêm số tuần phát sinh giao dịch chẳng hạn :).\
Hay số tuần liên tiếp phát sinh giao dịch

In [ ]:
recharge

In [ ]:
tbl_sum_money_C = recharge[recharge.COL_30 == 'C'].groupby('msisdn').sum().add_prefix('sum_money_C_')

In [ ]:
tbl_sum_money_C

In [ ]:
tbl_sum_money_C.drop(columns=['sum_money_C_week_of_year',	'sum_money_C_day_in_month'], inplace= True)

In [ ]:
tbl_sum_money_V = recharge[recharge.COL_30 == 'V'].groupby('msisdn').sum().add_prefix('sum_money_V_')

In [ ]:
tbl_sum_money_V.drop(columns=['sum_money_V_week_of_year',	'sum_money_V_day_in_month'], inplace= True)

In [ ]:
tbl_sum_money_V

In [ ]:
tbl_recharge = tb_recharge_service.join(tbl_sum_money_C)

In [ ]:
tbl_recharge = tbl_recharge.join(tbl_sum_money_V)

In [ ]:
tbl_recharge.sum_money_C_COL_31.fillna(0, inplace=True)
tbl_recharge.sum_money_V_COL_31.fillna(0, inplace=True)

In [ ]:
tbl_recharge['sum_money_C_V_COL_31'] = tbl_recharge.sum_money_C_COL_31 + tbl_recharge.sum_money_V_COL_31

In [ ]:
tbl_recharge.sample(10)

In [ ]:
summary_missing(tbl_recharge)

In [ ]:
# tbl_recharge.to_csv('recharge_features_extraction_25_8_2020.csv')

### ARPU_TRAIN ARPU _TEST

In [ ]:
arpu_train

In [ ]:
set(arpu_train.msisdn.unique()).intersection(set(arpu_test.msisdn.unique()))

In [ ]:
len(set(temp.PARTNER_MSISDN.unique()))

In [ ]:
len(set(temp.msisdn.unique()))

In [ ]:
len(set(temp.PARTNER_MSISDN.unique()).intersection(arpu_train.msisdn.unique()))

In [ ]:
len(set(temp.PARTNER_MSISDN.unique()).intersection(arpu_test.msisdn.unique()))

In [ ]:
arpu_train.iloc[0].COL_17

In [ ]:
arpu_train.sample(5).COL_17.values

#### convert COL_17 COL_18 to date time

In [ ]:
arpu_train['COL_17'] = pd.to_datetime(arpu_train['COL_17'], unit='ms')
arpu_train['COL_18'] = pd.to_datetime(arpu_train['COL_18'], unit='ms')

In [ ]:
arpu_test['COL_17'] = pd.to_datetime(arpu_test['COL_17'], unit='ms')
arpu_test['COL_18'] = pd.to_datetime(arpu_test['COL_18'], unit='ms')

In [ ]:
arpu_train

In [ ]:
arpu_train_feature = pd.DataFrame()
arpu_test_feature = pd.DataFrame()
arpu_train_feature['msisdn'] = arpu_train['msisdn']
arpu_test_feature['msisdn'] = arpu_test['msisdn']
arpu_train_feature.set_index('msisdn', inplace=True)
arpu_test_feature.set_index('msisdn', inplace=True)
arpu_train_feature = arpu_train_feature.join(arpu_train.set_index('msisdn').label)

In [ ]:
arpu_train.select_dtypes(include=['float64']).diff()

In [ ]:
_number_unique_by_col(arpu_train)

In [ ]:
arpu_train.dtypes

In [ ]:
summary_missing(arpu_train)

In [ ]:
print(f"Cac cot so norm:{list(_number_unique_by_col(arpu_train).index[8:].values)}")

In [ ]:
arpu_train.COL_21.value_counts()

Cac cot so norm:['COL_19', 'COL_20', 'COL_21', 'COL_22', 'COL_27a', 'COL_27b', 'COL_27c', 'COL_27d']

Cột 16 là giới tính thì onehot dùng crossbar

Cột 17 18 hình như là thời gian

Cột 14 thì tính tuổi là 2019 - đi năm rồi bining theo độ tuổ đi học đi làm?????

Cột 15 mà mã tỉnh thành thì phải (63 giá trij) => 

Cột 13 là cả tỉnh cả huyện (xã ) (710 giá trij) => 

Cả 13 và 15 là dữ liệu category những khá là rời rạc, có nhiều giá trị
nếu mà onehot trục tiếp thìn hiều feature vl
=> Có thể thống kê xem các mã trên gắn với nhãn 1 rồi bining theo nó để giảm số feature onehot tạo ra

In [ ]:
arpu_train_gender = pd.crosstab(arpu_train['msisdn'],arpu_train['COL_16'].fillna('missing'), dropna = True).rename_axis(None,axis=1).add_prefix('COL_16_')
arpu_test_gender = pd.crosstab(arpu_test['msisdn'],arpu_test['COL_16'].fillna('missing'), dropna = True).rename_axis(None,axis=1).add_prefix('COL_16_')

In [ ]:
arpu_train_gender.drop(columns=['COL_16_missing'], inplace=True)
arpu_test_gender.drop(columns=['COL_16_missing'], inplace=True)

In [ ]:
arpu_train_gender

In [ ]:
arpu_test_gender

In [ ]:
summary_missing(arpu_train_gender)

Missing hiện tại đang ứng với COL_16_F	COL_16_M là 0 0
=> nên để thấy hay để missing sẽ tốt hơn nhỉ???????

In [ ]:
arpu_train_feature['age'] =  (2019 - arpu_train.COL_14).values
arpu_test_feature['age'] =  (2019 - arpu_test.COL_14).values

In [ ]:
len((2019 - arpu_train.COL_14).values)

In [ ]:
arpu_train_feature[arpu_train_feature.age.notna()]

In [ ]:
arpu_train_feature.age.value_counts().head(50)

In [ ]:
for col in ['COL_15', 'COL_13']:
  stick_to_lb = _per_categorical(arpu_train, col)
  stick_to_lb.to_csv(col + '_stick_to_lb.csv')

In [ ]:
_per_categorical(arpu_train_feature, 'age').describe()

In [ ]:
arpu_train_feature[arpu_train_feature.age == 88]

In [ ]:
_per_categorical(arpu_train_feature, 'age').to_csv('age_stick_lb.csv')

In [ ]:
_per_categorical(arpu_train_feature, 'age').sort_index().plot(figsize=(10,10), xticks = [10,15,18,22,25,30,40,45,60,70,75,80,90,100])

In [ ]:
# [10,15,18,22,25,30,40,45,60,70,75,80,90,100]

In [ ]:
len(arpu_train_feature.age.unique()) - 1

In [ ]:
sorted(arpu_train_feature.age.dropna().unique())[0], sorted(arpu_train_feature.age.dropna().unique())[-1]

In [ ]:
age_bins = pd.IntervalIndex.from_tuples([(10, 15), (15, 18), (18, 22), (22, 25), (25, 30),
                                         (30, 40), (40, 45), (45, 60), (60,70), (70, 75),
                                         (75, 80), (80, 90), (90, 100)])
pd.cut(sorted(arpu_train_feature.age.dropna().unique()), age_bins)

In [ ]:
arpu_train_feature['age_bins'] = pd.cut(arpu_train_feature.age, age_bins)
arpu_test_feature['age_bins'] = pd.cut(arpu_test_feature.age, age_bins)

In [ ]:
for i in arpu_train_feature.age_bins.unique():
  print(i)

In [ ]:
for i in arpu_test_feature.age_bins.unique():
  print(i)

In [ ]:
_plot_per_categorical(arpu_train_feature, 'age_bins')

In [ ]:
COL_15_type = _devide_group(arpu_train,col='COL_15')
# COL_15_type

In [ ]:
COL_15_type

In [ ]:
def _map_lambda_cats(cols_list, colname, x): 
    cats = list(map(lambda x:colname + '_' + str(x), np.arange(len(cols_list)).tolist()))
    for i in range(len(cols_list)):
        if x in cols_list[i]:
            return cats[i]
        
def _map_cats(cols_list, colname, dataset):                    
    return list(map(lambda x: _map_lambda_cats(cols_list, colname, x), 
                    dataset[colname]))

arpu_train_feature['COL_15_TYPE'] = _map_cats(COL_15_type, 'COL_15', arpu_train)
arpu_test_feature['COL_15_TYPE'] = _map_cats(COL_15_type, 'COL_15', arpu_test)
print(pd.Series.unique(arpu_train_feature['COL_15_TYPE']))
print(pd.Series.unique(arpu_test_feature['COL_15_TYPE']))

In [ ]:
arpu_train_feature

In [ ]:
arpu_test_feature

In [ ]:
arpu_test

In [ ]:
summary_missing(arpu_train_feature)

In [ ]:
summary_missing(arpu_test_feature)

In [ ]:
arpu_train_feature = arpu_train_feature.join(arpu_train.set_index('msisdn')[['COL_17', 'COL_18', 'COL_19', 'COL_20', 'COL_21', 'COL_22', 'COL_27a', 'COL_27b', 'COL_27c', 'COL_27d']])
arpu_test_feature = arpu_test_feature.join(arpu_test.set_index('msisdn')[['COL_17', 'COL_18', 'COL_19', 'COL_20', 'COL_21', 'COL_22', 'COL_27a', 'COL_27b', 'COL_27c', 'COL_27d']])

In [ ]:
arpu_train_gender

In [ ]:
arpu_train_feature = arpu_train_feature.join(arpu_train_gender)

In [ ]:
arpu_test_feature = arpu_test_feature.join(arpu_test_gender)

In [ ]:
arpu_train_feature.head()

In [ ]:
arpu_test_feature.head()

In [ ]:
# arpu_train_feature.to_csv('arpu_train_feature_raw.csv')

In [ ]:
# arpu_test_feature.to_csv('arpu_test_feature_raw.csv')

In [ ]:
arpu_train.COL_17.describe()

In [ ]:
arpu_train.COL_18.describe()

In [ ]:
(arpu_train.COL_17 - arpu_train.COL_18)[57720]

In [ ]:
arpu_train_feature_old = pd.read_csv('arpu_train_feature_raw.csv')

In [ ]:
arpu_test_feature_old = pd.read_csv('arpu_test_feature_raw.csv')

In [ ]:
arpu_train

In [ ]:
arpu_train.COL_21.unique()

In [ ]:
arpu_train[arpu_train.COL_21 == 0]

In [ ]:
arpu_train[arpu_train.COL_21 != 0]

In [ ]:
arpu_train_feature_old.set_index('msisdn', inplace=True)
arpu_test_feature_old.set_index('msisdn', inplace=True)
arpu_train.set_index('msisdn', inplace=True)
arpu_test.set_index('msisdn', inplace=True)

arpu_train

In [ ]:
arpu_train.COL_18.dt.to_period('M')

In [ ]:
COL_17_year = arpu_train.COL_17.dt.year.to_frame(name='COL_17_year')
COL_18_year = arpu_train.COL_18.dt.year.to_frame(name='COL_18_year')
COL_17_month_year = arpu_train.COL_17.dt.to_period('M').to_frame(name='COL_17_month_year')
COL_18_month_year = arpu_train.COL_18.dt.to_period('M').to_frame(name='COL_18_month_year')

In [ ]:
COL_18_month_year.loc['0001fa0c3d21c0d758ec'].COL_18_month_year

In [ ]:
pd.to_datetime('2019-10').to_period('M')

In [ ]:
(pd.to_datetime('2019-10').to_period('M') - COL_18_month_year.loc['0001fa0c3d21c0d758ec'].COL_18_month_year)

In [ ]:
(pd.to_datetime('2019-10').to_period('M') - COL_18_month_year.loc['0001fa0c3d21c0d758ec'].COL_18_month_year).n

#### Thoi diem thu thap tat ca cac bang loan, reacharge, temp deu la trong thang 10/2019 => coi nhu la current time chung

In [ ]:
COL_17_up_to_now = (pd.to_datetime('2019-10').to_period('M') - COL_17_month_year.COL_17_month_year).to_frame(name='COL_17_upto_2019_10')
COL_18_up_to_now = (pd.to_datetime('2019-10').to_period('M') - COL_18_month_year.COL_18_month_year).to_frame(name='COL_18_upto_2019_10')

In [ ]:
COL_17_up_to_now

In [ ]:
def get_num_from_month_end_COL_17(month_end):
  return month_end.COL_17_upto_2019_10.n

def get_num_from_month_end_COL_18(month_end):
  return month_end.COL_18_upto_2019_10.n

In [ ]:
COL_17_up_to_now = COL_17_up_to_now[COL_17_up_to_now.COL_17_upto_2019_10.notnull()].apply(get_num_from_month_end_COL_17, axis= 1).to_frame(name='COL_17_upto_2019_10')
COL_18_up_to_now = COL_18_up_to_now[COL_18_up_to_now.COL_18_upto_2019_10.notnull()].apply(get_num_from_month_end_COL_18, axis= 1).to_frame(name='COL_18_upto_2019_10')

In [ ]:
arpu_train_feature_old = arpu_train_feature_old.join(COL_17_year).join(COL_18_year).join(COL_17_month_year).join(COL_18_month_year).join(COL_17_up_to_now).join(COL_18_up_to_now)

In [ ]:
arpu_train_feature_old

arpu test

In [ ]:
COL_17_year = arpu_test.COL_17.dt.year.to_frame(name='COL_17_year')
COL_18_year = arpu_test.COL_18.dt.year.to_frame(name='COL_18_year')
COL_17_month_year = arpu_test.COL_17.dt.to_period('M').to_frame(name='COL_17_month_year')
COL_18_month_year = arpu_test.COL_18.dt.to_period('M').to_frame(name='COL_18_month_year')

In [ ]:
COL_17_up_to_now = (pd.to_datetime('2019-10').to_period('M') - COL_17_month_year.COL_17_month_year).to_frame(name='COL_17_upto_2019_10')
COL_18_up_to_now = (pd.to_datetime('2019-10').to_period('M') - COL_18_month_year.COL_18_month_year).to_frame(name='COL_18_upto_2019_10')

In [ ]:
COL_17_up_to_now = COL_17_up_to_now[COL_17_up_to_now.COL_17_upto_2019_10.notnull()].apply(get_num_from_month_end_COL_17, axis= 1).to_frame(name='COL_17_upto_2019_10')
COL_18_up_to_now = COL_18_up_to_now[COL_18_up_to_now.COL_18_upto_2019_10.notnull()].apply(get_num_from_month_end_COL_18, axis= 1).to_frame(name='COL_18_upto_2019_10')

In [ ]:
arpu_test_feature_old = arpu_test_feature_old.join(COL_17_year).join(COL_18_year).join(COL_17_month_year).join(COL_18_month_year).join(COL_17_up_to_now).join(COL_18_up_to_now)

In [ ]:
arpu_test_feature_old

In [ ]:
ls

In [ ]:
# arpu_train_feature_old.to_csv('arpu_train_feature_raw_ver_3.csv')
# arpu_test_feature_old.to_csv('arpu_test_feature_raw_ver_3.csv')

#### Vãi sao lại lẫn data 2020 trong file arpu train thế này 0001fa0c3d21c0d758ec

In [ ]:
arpu_train.loc['0001fa0c3d21c0d758ec']

In [ ]:
arpu_train.columns

COL_16_F	COL_16_M la 0 0 tuong ung voi cot gender la null

In [ ]:
arpu_train_feature_old[arpu_train_feature_old.COL_16_F.isna()]

# Nháp :))

### sum nan behavior: group by ignore nan, nhưng sum không ignore, có hmà khác thì lại ignore tùy :)) => nên cẩn thận-

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.DataFrame({'a': [1, 2, 3, 5], 'b': [4, np.NaN, 6, 4]})

In [ ]:
df2 = pd.DataFrame({'a': [np.NaN, np.NaN, np.NaN, np.NaN], 'b': [4, np.NaN, 6, 4]})

In [ ]:
df

In [ ]:
df.groupby(['b']).sum()

In [ ]:
df2

In [ ]:
df2.groupby(['b']).sum()

In [ ]:
df1 = pd.DataFrame({'a':range(6),
                    'b':[5,3,6,9,2,4]}, index=list('abcdef'))

print (df1)

df2 = pd.DataFrame({'c':range(4),
                    'd':[10,20,30, 40]}, index=list('bhia'))

print (df2)



In [ ]:
df4 = df2.join(df1)

In [ ]:
df4

In [ ]:
df2

### Sum nan value?

In [ ]:
df = pd.DataFrame({'a': ['1', '1', '3', '3'], 'b': [np.NaN, np.NaN, 0, 0]}, index=list('abcd'))

In [ ]:
df

In [ ]:
df.groupby('a')['b'].agg(sum_b = ('b', 'sum'))

In [ ]:
df.groupby('a')['b'].sum(min_count = 0)

In [ ]:
df.groupby('a')['b'].sum(min_count = 1)

pivot table, group by, crossbar => agg dữ liệu

biểu diễn mức giữa  pivot table+ có nhiều thao tác phức tạp => group by

Bẻ dữ liệu dọc (các loại category) => thành các cột (thanh ngang)) dùng crossbar

Khi tổng hợp dữ liệu - agg cần chứ ý behavior của các operator, hàm, sử dụng để tổng hợp ứng xử với giá trị null

Ví dụ groupby mặc định bỏ qua null (cột dùng để groupby nếu có giá trị null thì nó sẽ bỏ hàng có giá trị null của cột đó tuy nhiên hàng đó vẫn có thể có dữ liệu ở các cột khác hoặc vì lí do nào đó vẫn muốn có nó thì phải xử lí trước fill missing chẳng hạn ? để groupby)

Sau khi groupby xong dùng các hàm agg như sum, mean, pd.Series.unique() chẳng hạn cũng cần chú ý đến behavior của nó. Chẳng hạng lệnh sum sẽ coi các giá trị null của cột nó tính sum là bằng 0 để tính sum => chẳng hạn có 2 id sau khi group by xong theo id, tính sum của cột A ứng với 2 id này. id đầu tiên cột A của nó toàn giá trị null, cột A thứ 2 thì cột A của nó toàn giá tri 0 => dùng lệnh sum thì cả 2 id đều có kq cột A là giá trị 0 mặc dù cột A của id đầu toàn null (missing dữ liệu) còn id 2 là có giá trị nhưng là giá trị 0 => 2 tính chất khác nhau đang được được coi là gióng nhau => KHÔNG TỐT. Tương tự cần chú ý các hàm agg khác là có bỏ qua null, hay cho null là 0... => behavior của nó ảnh hưởng trực tiếp đến bài toán.


Việc xư lí dữ liệu missing:
Không phải lúc nào fill missing cũng tốt còn tùy thuộc vào bài toán và mô hình ta sử dụng. Có model chỉ lấy giá trị số, và không xử lí nếu có missin, có model thì xử lí cả missing, có model xử lí cả số và category.....
Việc đầu tiên khi xử lí dữ liệu missing là xác định xem tính chất của dữ liệu missing đó là gì:

- null/nat/nan => missing data của dữ liệu số, của dữ liệu ngày tháng...
- missing là do ngẫu nhiên hay do có chủ đích chẳng hạn báo là dữ liệu thu thập trong ngày đó, thời điểm đó, của người/sự vật đó là không có s-> là do người đó không thực hiện hành đọng này..... ( có nhiều loại missing và ý nghĩa => cần tìm hiểu theem)
=> Tùy thuộc vào bản chất mà xừ lí. Không phải giá trị nào null fill cũng tốt. nếu cột đó toàn là catogory và mô hình nhận cả category thì coi missing là một loại category luôn cũng được. nếu mô hình không nhận categor bắt nhận giá trị số thì có thể biến cột đó thành nhiều cột khác nhau ứng với category trong đó có cột tương ứng với gía trị missing => tóm lại có bài toán mà missing là có ý nghĩa riêng của nó việc fill nó làm mất đi ý nghĩa vốn có.
Cũng có các cột số ta cũng có thể coi là category => crossbar bẻ nó thành các cột khác nhau ứng với các "category"

Nếu range số nó lớn quá thì thế nào => có thể bining ra rồi coi là category.

Khi xử lí category cần xem category đó bản chất không phân biệt thứ tự hay có phân biệt thứ tự... tùy thuộc vào nó để sử dụng các phép xử lí khác nhau onehot hay label coding hay coi ngược lại là số.....


NGoài ra còn các technique như getdummies, ......

In [ ]:
df1 = pd.DataFrame([{'a':1,'b':2},{'a':1,'b':3},{'a':2,'b':4}],
                   columns = ['a','b']).set_index('a')
df2 = pd.DataFrame([{'a':1,'c':5},{'a':2,'c':6},{'a':3, 'c':7}, {'a':2, 'c':np.nan}, {'a':9, 'c':10}],columns = ['a','c']).set_index('a')

In [ ]:
df1

In [ ]:
df2

In [ ]:
df1.merge(df2, left_index=True, right_index=True)


In [ ]:
pd.concat([df1,df2], join='inner', axis=1)

In [ ]:
df = pd.DataFrame([[0, 0, 0],
                   [0, 10, 0],
                   [4, 0, 0],
                   [1, 2, 3]],
                  columns=['first', 'second', 'third'])

In [ ]:
df

DataFrame.gt(other, axis='columns', level=None)[source]
Get Greater than of dataframe and other, element-wise (binary operator gt).

Among flexible wrappers (eq, ne, le, lt, ge, gt) to comparison operators.

Equivalent to ==, =!, <=, <, >=, > with support to choose axis (rows or columns) and level for comparison.

In [ ]:
df.where(df.gt(0)).min(axis = 1)

In [ ]:
df[df.gt(0)].min(axis = 1)

In [ ]:
df[df > 0].min(axis = 1)